# Single-Network Vertex Nomination

## References

theory papers:
- theory of consistency and Bayes optimality: On consistent vertex nomination schemes
- dealing with corrupted networks (e.g., not one-to-one correspondence): Vertex Nomination, Consistent Estimation, and Adversarial Modification

applications papers:
- social networks: Vertex nomination via local neighborhood matching
- data associated with human trafficking: Vertex nomination schemes for membership prediction

this is the spectral vertex nomination stuff? 
https://microsoft.github.io/graspologic/tutorials/vertex_nomination/SpectralVertexNomination.html

## Summary

1. start with an n by n graph G and a vertex of interest v*;
2. pick your favorite embedding method to get n points in R^{d} (each d-dimensional vector is to be interpreted as a representation of a vertex in G)
3. then, depending on the embedding method you use, pick a distance / dissimilarity on R^{d} (for instance, if i suspect my graph is generated from an SBM and i embed the graph using ASE / LSE i may choose either Euclidean distance for simplicity or Mahalanobis distance to exploit the assumed structure of the point in R^{d});
4. calculate the distances between the vector corresponding to v* and vectors corresponding to the other n - 1 vertices;
5. return the closest k vertices to v* says your selected distance;

Say you're a criminal investigator trying to uncover a human trafficking ring. You build a network of potential suspects: some of the nodes in the network represent human traffickers, and some represent innocent people. The edges of the network represent a working relationship between a given pair of individuals.

Your team has limited resources, and so it's difficult to scrutinize everybody in the network directly to see if they are human traffickers. Ideally, you'd like to use your network to nominate potential suspects, so that you can prioritize your investigative efforts. Your goal, then, is to build an ordered list of nodes in the network that are most similar to the nodes you already know belong to human traffickers. Ideally, the first nodes in the list would be more likely to be traffickers, and the nodes would get less and less likely the further down in the list you go.

This is the idea behind *single-network vertex nomination*. You have a group of "seed nodes" that you know have the properties you're looking for, and then you take the rest of the nodes in your network and rank them by the probability that they share those properties. The nomination task here isn't just classification: it's prioritization. You're prioritizing how important the rest of your nodes are with respect to the seed nodes, with the most important nodes at the top.

There are a few approaches to vertex nomination, but what we'll focus on is *spectral vertex nomination*. The general idea is that you embed your network, and then you just order the nodes by how close they are to your seed nodes.